In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from tqdm.notebook import tqdm

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import os
os.environ['LOGURU_LEVEL'] = 'INFO'

In [ ]:
# Laad data
# Gem. ed tussen src en tgt
# Laad model
# Doe predictions
# Gem. ed tussen src en tgt
# Genereer icdar output en doe voorspelling en voeg suggesties in voor alle tokens van max 10 tekens

In [ ]:
in_dir = Path('icdar-dataset-20220207')

train = pd.read_csv(in_dir/'task2_train.csv', index_col=0)
val = pd.read_csv(in_dir/'task2_val.csv', index_col=0)
test = pd.read_csv(in_dir/'task2_test.csv', index_col=0)

train = train.fillna('')
val = val.fillna('')
test = test.fillna('')

In [ ]:
print('train:', train.shape[0], 'samples')
print('val:', val.shape[0], 'samples')
print('test:', test.shape[0], 'samples')

In [ ]:
def add_lens(data: pd.DataFrame) -> pd.DataFrame:
    data['len_ocr'] = data['ocr'].apply(lambda x: len(x))
    data['len_gs'] = data['gs'].apply(lambda x: len(x))

    return data

train = add_lens(train)
val = add_lens(val)
test = add_lens(test)

In [ ]:
from torch.utils.data import Dataset

class Task2Dataset(Dataset):
    def __init__(self, data, max_len=11):
        self.ds = data.query(f'len_ocr < {max_len}').query(f'len_gs < {max_len}').copy()
        self.ds = self.ds.reset_index(drop=True)

    def __len__(self):
        return self.ds.shape[0]

    def __getitem__(self, idx):
        sample = self.ds.iloc[idx]
        return [char for char in sample.ocr], [char for char in sample.gs]

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

vocab_transform = {}

def yield_tokens(data, col):
    for token in data[col].to_list():
        for char in token:
            yield char

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for name in ('ocr', 'gs'):
    
    vocab_transform[name] = build_vocab_from_iterator(yield_tokens(train, name),
                                                      min_freq=1,
                                                      specials=special_symbols,
                                                      special_first=True)
# Set UNK_IDX as the default index. This index is returned when the token is not found.
# If not set, it throws RuntimeError when the queried token is not found in the Vocabulary.
for name in ('ocr', 'gs'):
  vocab_transform[name].set_default_index(UNK_IDX)

In [ ]:
from typing import Iterable, List

from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for name in ('ocr', 'gs'):
    text_transform[name] = sequential_transforms(vocab_transform[name],  # Numericalization (char -> idx)
                                                 tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform['ocr'](src_sample))
        tgt_batch.append(text_transform['gs'](tgt_sample))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch.to(torch.int64), tgt_batch.to(torch.int64)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    def forward(self, input, hidden):
        # print('Encoder')
        # print('input size', input.size())
        # print('hidden size', hidden.size())
        embedded = self.embedding(input) 
        # print('embedded size', embedded.size())
        # print(embedded)
        # print('embedded size met view', embedded.view(1, 1, -1).size())
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=11):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)

        # print('embedded size', embedded.size())
        # print(embedded)
        embedded = torch.permute(embedded, (1, 0, 2))
        # print('permuted embedded size', embedded.size())
        # print(embedded)

        # print('hidden size', hidden.size())
        # print(hidden)

        # print('permuted embedded[0] size', embedded[0].size())
        # print('hidden[0] size', hidden[0].size())

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)

        # print('attn_weights', attn_weights.size())
        # print('attn_weights unsqueeze(1)', attn_weights.unsqueeze(1).size())
        # print('encoder outputs', encoder_outputs.size())


        attn_applied = torch.bmm(attn_weights.unsqueeze(1),
                                 encoder_outputs)

        # print('attn_applied', attn_applied.size())
        # print('attn_applied squeeze', attn_applied.squeeze().size())
        output = torch.cat((embedded[0], attn_applied.squeeze()), 1)
        # print('output', output.size())
        output = self.attn_combine(output).unsqueeze(0)
        # print('output', output.size())

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)

        # print(f'output: {output.size()}; hidden: {hidden.size()}; attn_weigts: {attn_weights.size()}')

        return output, hidden, attn_weights

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [ ]:
class ICDARTask2Seq2seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout, max_length):
        super(ICDARTask2Seq2seq, self).__init__()

        self.encoder = EncoderRNN(input_size, hidden_size)
        self.decoder = AttnDecoderRNN(hidden_size, output_size, 
                                      dropout_p=dropout, max_length=max_length)
    
    def forward(self, input, encoder_hidden, target, max_length):
        # input is src seq len x batch size
        # input voor de encoder (1 stap) moet zijn input seq len x batch size x 1
        input_tensor = input.unsqueeze(2)
        # print('input tensor size', input_tensor.size())

        input_length = input.size(0)

        batch_size = input.size(1)

        # Encoder part
        encoder_outputs = torch.zeros(batch_size, max_length, self.encoder.hidden_size, 
                                      device=device)
        # print('encoder outputs size', encoder_outputs.size())
    
        for ei in range(input_length):
            # print(f'Index {ei}; input size: {input_tensor[ei].size()}; encoder hidden size: {encoder_hidden.size()}')
            encoder_output, encoder_hidden = self.encoder(
                input_tensor[ei], encoder_hidden)
            # print('Index', ei)
            # print('encoder output size', encoder_output.size())
            # print('encoder outputs size', encoder_outputs.size())
            # print('output selection size', encoder_output[:, 0].size())
            # print('ouput to save', encoder_outputs[:,ei].size())
            encoder_outputs[:, ei] = encoder_output[0, 0]
        
        # print('encoder outputs', encoder_outputs)
        # print('encoder hidden', encoder_hidden)

        # Decoder part
        # Target = seq len x batch size
        # Decoder input moet zijn: batch_size x 1 (van het eerste token = BOS)
        target_length = target.size(0)

        decoder_input = torch.tensor([[BOS_IDX] for _ in range(batch_size)], device=device)
        # print('decoder input size', decoder_input.size())

        decoder_outputs = torch.zeros(batch_size, max_length, self.decoder.output_size, 
                                      device=device)

        decoder_hidden = encoder_hidden

        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = self.decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.detach()  # detach from history as input

            # print(f'Index {di}; decoder output size: {decoder_output.size()}; decoder input size: {decoder_input.size()}')
            decoder_outputs[:, di] = decoder_output

        # Zero out probabilities for padded chars
        target_masks = (target != PAD_IDX).float()

        # Compute log probability of generating true target words
        # print('P (decoder_outputs)', decoder_outputs.size())
        # print(target.transpose(0, 1))
        # print('Index', target.size(), target.transpose(0, 1).unsqueeze(-1))
        target_gold_std_log_prob = torch.gather(decoder_outputs, index=target.transpose(0, 1).unsqueeze(-1), dim=-1).squeeze(-1) * target_masks.transpose(0, 1)
        #print(target_gold_std_log_prob)
        scores = target_gold_std_log_prob.sum(dim=1)

        #print(scores)

        return scores, decoder_outputs


In [ ]:
from torch.utils.data import DataLoader

import edlib

def calculate_ed(src_strings, tgt_strings):
    dist = []

    for src, tgt in zip(src_strings, tgt_strings):
        #print(repr(src), repr(tgt))
        ed = edlib.align(src, tgt)
        dist.append(ed['editDistance'])

    return np.mean(dist), np.std(dist)


def indices2string(indices, itos):
    output = []
    for idxs in indices:
        #print(idxs)
        string = []
        for idx in idxs:
            if idx not in (UNK_IDX, PAD_IDX, BOS_IDX):
                if idx == EOS_IDX:
                    break
                else:
                    string.append(itos[idx])
        word = ''.join(string)
        output.append(word)
    return output


def ed_predicted_strings(model, dataloader, tgt_vocab, tgt_strings, DEVICE):
    itos = tgt_vocab.get_itos()
    output_strings = []

    with torch.no_grad():
        for src, tgt in dataloader:
            src = src.to(DEVICE)
            tgt = tgt.to(DEVICE)
            
            batch_size = src.size(1)

            encoder_hidden = model.encoder.initHidden(batch_size=batch_size)

            example_losses, decoder_ouputs = model(src, encoder_hidden, tgt, MAX_LENGTH)
            example_losses = -example_losses
            batch_loss = example_losses.sum()
            # Generate string outputs
            output_idxs = decoder_ouputs.argmax(-1)
            #print(output_idxs.size())
            #print(output_idxs)

            strings_batch = indices2string(output_idxs, itos)
            for s in strings_batch:
                output_strings.append(s)

    m, std = calculate_ed(output_strings, tgt_strings)

    return m, std, output_strings


def get_gold_tgt_words(dataset):
    src_strings = []
    tgt_strings = []

    for src, tgt in dataset:
        src_strings.append(''.join(src))
        tgt_strings.append(''.join(tgt))
    return src_strings, tgt_strings

In [ ]:
MAX_LENGTH = 11
BATCH_SIZE = 128

dataset = Task2Dataset(val)
src_strings, tgt_strings = get_gold_tgt_words(dataset)

In [ ]:
m, std = calculate_ed(src_strings, tgt_strings)
print(f'src -> tgt: m ed {m}, std {std}')

In [ ]:
def predict_and_calculate_ed(model, dataloader, tgt_vocab, tgt_strings, MAX_LENGTH, DEVICE):
    was_training = model.training
    model.eval()

    itos = tgt_vocab.get_itos()
    output_strings = []

    with torch.no_grad():
        for src, tgt in dataloader:
            src = src.to(DEVICE)
            tgt = tgt.to(DEVICE)
            
            batch_size = src.size(1)

            encoder_hidden = model.encoder.initHidden(batch_size=batch_size)

            example_losses, decoder_ouputs = model(src, encoder_hidden, tgt, MAX_LENGTH)

            # Generate string outputs
            output_idxs = decoder_ouputs.argmax(-1)
            #print(output_idxs.size())
            #print(output_idxs)

            strings_batch = indices2string(output_idxs, itos)
            for s in strings_batch:
                output_strings.append(s)

            m, std = calculate_ed(output_strings, tgt_strings)

    if was_training:
        model.train()

    return m, std, output_strings

In [ ]:
hidden_size = 256
PATH = '/Users/janneke/Google Drive/model.pt'

model = ICDARTask2Seq2seq(len(vocab_transform['ocr']), hidden_size, len(vocab_transform['gs']), 0.1, MAX_LENGTH)
model.load_state_dict(torch.load(PATH, map_location=device))
model.eval()

In [ ]:
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

m, ed, pred_strings = predict_and_calculate_ed(model, dataloader, vocab_transform['gs'], tgt_strings, MAX_LENGTH, device)

In [ ]:
m, std = calculate_ed(pred_strings, tgt_strings)
print(f'pred -> tgt: m ed {m}, std {std}')

In [ ]:
import random

for i in random.choices(range(len(src_strings)), k=5):
    print(f'> {src_strings[i]}\n= {tgt_strings[i]}\n< {pred_strings[i]}\n')

In [ ]:
def validate_model(model, dataloader, MAX_LENGTH, DEVICE):
    cum_loss = 0
    cum_examples = 0

    was_training = model.training
    model.eval()

    with torch.no_grad():
        for src, tgt in dataloader:
            src = src.to(DEVICE)
            tgt = tgt.to(DEVICE)
            
            batch_size = src.size(1)

            encoder_hidden = model.encoder.initHidden(batch_size=batch_size)

            example_losses, decoder_ouputs = model(src, encoder_hidden, tgt, MAX_LENGTH)
            example_losses = -example_losses
            batch_loss = example_losses.sum()

            bl = batch_loss.item()
            cum_loss += bl
            cum_examples += batch_size

    if was_training:
        model.train()

    return cum_loss/cum_examples

In [ ]:
validate_model(model, dataloader, MAX_LENGTH, device)

## Create icdar results

* Create icdar results data structure with all strings that need to be corrected
* The corrections will be added later

In [ ]:
from datautils import generate_data

in_dir = Path('../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish')
data_test, X_test = generate_data(in_dir)

In [ ]:
from collections import defaultdict

to_predict = defaultdict(list)
result = {}

for key, text in data_test.items():
    print(key, len(text.tokens))
    result[key] = {}
    for token in text.tokens:
        if token.ocr != token.gs:
            num_tokens = len(token.ocr.strip().split(' '))
            token_key = f'{token.start}:{num_tokens}'
            result[key][token_key] = {}
            to_predict[token.ocr.strip()].append({'text': key, 'token_key': token_key})
    break

In [ ]:
result

In [ ]:
to_predict

In [ ]:
ocr_strings = ['Zha I']
pred_strings = ['Zha']

for ocr, pred in zip(ocr_strings, pred_strings):
    for position in to_predict[ocr]:
        result[position['text']][position['token_key']][pred] = 1.0

In [ ]:
result